# Workout Data Workbook 
by Alex Guerrero

### Packages Imported

In [1]:
# from email import header
# import streamlit as st
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

### CSV File Import

In [2]:
filename = 'data/workout - log.csv'
workout_data = pd.read_csv(filename)

## Exploring of the data

In [3]:
# Printing the first 5 rows
workout_data.head()

,Date,Exercise,Muscle,Sets,Reps,Weight,Duration,Note
0,5/31/2021,Bodyweight,Weight,1,1.0,127.8,NaN,NaN
1,6/1/2021,Bodyweight,Weight,1,1.0,128.3,NaN,NaN
2,6/7/2021,Bodyweight,Weight,1,1.0,130.0,NaN,NaN
3,6/9/2021,Bodyweight,Weight,1,1.0,128.9,NaN,NaN
4,7/8/2021,Bodyweight,Weight,1,1.0,129.0,NaN,NaN


In [4]:
# Printing the last 5 rows
workout_data.tail()

,Date,Exercise,Muscle,Sets,Reps,Weight,Duration,Note
4192,12/11/2024,Leg Curls,Glutes_Hamstrings,3,10.0,90.0,NaN,NaN
4193,12/11/2024,Leg Curls,Glutes_Hamstrings,4,10.0,100.0,NaN,NaN
4194,12/11/2024,Seated Calf Raises,Calves,1,10.0,20.0,NaN,NaN
4195,12/11/2024,Seated Calf Raises,Calves,2,10.0,20.0,NaN,NaN
4196,12/11/2024,Seated Calf Raises,Calves,3,10.0,20.0,NaN,NaN


In [5]:
# date shape
workout_data.shape

(4197, 8)

In [6]:
# printing data columns
workout_data.columns

Index(['Date', 'Exercise', 'Muscle', 'Sets', 'Reps', 'Weight', 'Duration',
       'Note'],
      dtype='object')

In [7]:
# Dara more information
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4197 entries, 0 to 4196
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      4197 non-null   object 
 1   Exercise  4197 non-null   object 
 2   Muscle    4197 non-null   object 
 3   Sets      4197 non-null   int64  
 4   Reps      4196 non-null   float64
 5   Weight    2940 non-null   float64
 6   Duration  17 non-null     object 
 7   Note      223 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 262.4+ KB


### Dataframe Clean Up

In [8]:
# Convert Date to datetime
workout_data['Date'] = pd.to_datetime(workout_data['Date'], format="%m/%d/%Y")

# Ensure Exercise, Muscle, and Note are strings
workout_data['Exercise'] = workout_data['Exercise'].astype(str)
workout_data['Muscle'] = workout_data['Muscle'].astype(str)
workout_data['Note'] = workout_data['Note'].fillna("").astype(str)

# Convert Sets and Reps to float64
workout_data['Sets'] = workout_data['Sets'].fillna(0).astype(int)
workout_data['Reps'] = workout_data['Reps'].fillna(0).astype(int)

# Convert Weight to float64, keeping NaN for missing values
workout_data['Weight'] = workout_data['Weight'].astype(float)

# Convert Duration to object (string) with default value for missing data
workout_data['Duration'] = workout_data['Duration'].fillna("00:00:00").astype(str)

# Convert Total_volume to float64, keeping NaN for missing values
workout_data['Total_volume'] = workout_data['Reps'] * workout_data['Weight'] 
workout_data['Total_volume'] = workout_data['Total_volume'].astype(float)

In [9]:
# re-checking dataframe shape
workout_data.shape

(4197, 9)

In [10]:
workout_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4197 entries, 0 to 4196
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          4197 non-null   datetime64[ns]
 1   Exercise      4197 non-null   object        
 2   Muscle        4197 non-null   object        
 3   Sets          4197 non-null   int64         
 4   Reps          4197 non-null   int64         
 5   Weight        2940 non-null   float64       
 6   Duration      4197 non-null   object        
 7   Note          4197 non-null   object        
 8   Total_volume  2940 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 295.2+ KB


### Calculations

In [11]:
# Date Parameters -- Calculations
today = date.today()
start_of_week = today - relativedelta(days=today.weekday())
end_of_Week = start_of_week + relativedelta(days=6)
first_day_of_month = today.replace(day=1)
next_month_first_day = today + relativedelta(months=1, day=1)

# Formatting date parameters into string
today = today.strftime('%m-%d-%Y')
start_of_week = start_of_week.strftime('%m-%d-%Y')
end_of_Week = end_of_Week.strftime('%m-%d-%Y')
first_day_of_month = first_day_of_month.strftime('%m-%d-%Y')
next_month_first_day = next_month_first_day.strftime('%m-%d-%Y')

## KPIs

In [12]:
# total count of record in current week
weekly_count = len(workout_data.query(" Date >= @start_of_week & Date <= @end_of_Week")['Date'].unique())
print(f"Weekly Record Count: {weekly_count}")

Weekly Record Count: 0


In [13]:
# total count of record in current month
monthly_count = len(workout_data.query(" Date >= @first_day_of_month & Date < @next_month_first_day")['Date'].unique())
print(f"Montly Record Count: {monthly_count}")

Montly Record Count: 0


In [14]:
# unique date total record count
total_records_count = len(workout_data.Date.unique())
print(f"Total record count: {total_records_count}")

Total record count: 437


In [15]:
# total volume per day
total_volume_per_day = workout_data[["Date","Total_volume"]].groupby(by="Date").sum("Total_volume")
total_volume_per_day.reset_index()

,Date,Total_volume
0,2021-05-31,127.8
1,2021-06-01,128.3
2,2021-06-07,130.0
3,2021-06-09,128.9
4,2021-07-08,129.0
...,...,...
432,2024-12-03,137.3
433,2024-12-04,8580.0
434,2024-12-05,0.0
435,2024-12-09,5520.0


### Latest Record

In [16]:
latest_record = workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]
latest_record

,Date,Exercise,Muscle,Sets,Reps,Weight,Duration,Note,Total_volume
4181,2024-12-11,Walk,Cardio,1,1,NaN,0:10:00,,NaN
4182,2024-12-11,Barbell Squats,Quadriceps,1,10,45.0,00:00:00,,450.0
4183,2024-12-11,Barbell Squats,Quadriceps,2,10,115.0,00:00:00,2 x 35,1150.0
4184,2024-12-11,Barbell Squats,Quadriceps,3,10,115.0,00:00:00,2 x 35,1150.0
4185,2024-12-11,Barbell Squats,Quadriceps,4,10,95.0,00:00:00,2 x 25,950.0
4186,2024-12-11,Leg Extensions,Quadriceps,1,10,40.0,00:00:00,single leg,400.0
4187,2024-12-11,Leg Extensions,Quadriceps,2,15,70.0,00:00:00,both legs,1050.0
4188,2024-12-11,Leg Extensions,Quadriceps,3,10,50.0,00:00:00,single leg,500.0
4189,2024-12-11,Leg Extensions,Quadriceps,4,10,90.0,00:00:00,both legs,900.0
4190,2024-12-11,Leg Curls,Glutes_Hamstrings,1,15,70.0,00:00:00,,1050.0


In [17]:
print(f"Recent muscle groups worked out: {workout_data[workout_data['Date'] == workout_data['Date'].unique()[-1]]['Muscle'].unique()}")


Recent muscle groups worked out: ['Cardio' 'Quadriceps' 'Glutes_Hamstrings' 'Calves']


In [18]:
# what muscle did we work out last
for unique_muscle in latest_record['Muscle'].unique():
    print(unique_muscle)

Cardio
Quadriceps
Glutes_Hamstrings
Calves
